# Проект 3. Решить задачу DaNetQA / BoolQ

Можно решить как задачу для русского, так и для английского.

Либо провести эксперименты с многоязычной моделью

https://russiansuperglue.com/ru/tasks/task_info/DaNetQA

## Описание
Причинно-следственная связь, логический вывод, Natural Language Inference

DaNetQA - это набор да/нет вопросов с ответами и фрагментом текста, содержащим ответ. Все вопросы были написаны авторами без каких-либо искусственных ограничений.

Каждый пример представляет собой триплет (вопрос, фрагмент текста, ответ) с заголовком страницы в качестве необязательного дополнительного контекста.

Настройка классификации текстовых пар аналогична существующим задачам логического вывода (NLI)

### Тип задачи
Логика, Commonsense, Знания о мире. Бинарная классификация: true/false

# Imports

In [22]:
import pandas as pd
import numpy as np

# Подготовка данных

### Загрузка данных

In [23]:
def loadJSONL(path, name):
    df = pd.read_json(path, lines=True)
    df.drop('idx', axis=1, inplace=True)
    print(name)
    display(df.head())
    if (df.columns.values == 'label').any():
        s = np.unique(df['label'].to_numpy(), return_counts=True)[1]
        print(f"True answer: {s[1]}")
        print(f"False answer: {s[0]}")
        print("")
    return df

In [24]:
df_train = loadJSONL("DaNetQA/train.jsonl", "Train set")
df_validation = loadJSONL("DaNetQA/val.jsonl", "Validation set")
df_test = loadJSONL("DaNetQA/test.jsonl", "Test set:")


Train set


,question,passage,label
0,Вднх - это выставочный центр?,«Вы́ставочный центр» — станция Московского мон...,True
1,Вднх - это выставочный центр?,"Вы́ставка достиже́ний наро́дного хозя́йства ,...",True
2,Был ли джиган в black star?,Вместе с этим треком они выступили на церемони...,True
3,Xiaomi конкурент apple?,"Xiaomi — китайская компания, основанная в 2010...",True
4,Был ли автомат калашникова в вов?,Отметив некоторые недостатки и в целом удачную...,False


True answer: 1061
False answer: 688

Validation set


,question,passage,label
0,Есть ли вода на марсе?,Гидросфера Марса — это совокупность водных зап...,True
1,Состоит ли англия в евросоюзе?,В полночь с 31 января на 1 февраля 2020 года п...,False
2,Действительно ли в ссср не было адвокатов?,Семён Львович Ария — советский и российский ю...,False
3,Была ли чума в оране?,"Чума — это и абсурд, что осмысливается как фор...",True
4,Был ли кетчуп в читосе?,Текущий каталог продукции размещен на сайте пр...,True


True answer: 412
False answer: 409

Test set:


,question,passage
0,Полезна ли ртуть с градусника?,"Отравления ртутью — расстройства здоровья, св..."
1,Являются ли сапрофаги хищниками?,Фауна лесных почв — совокупность видов животны...
2,Водятся ли в индии крокодилы?,"Болотный крокодил, или магер — пресмыкающееся..."
3,Есть ли в батате крахмал?,"Клубневидно вздутые корни весят до 15 кг, сод..."
4,Был ли человек в железной маске?,Остров Сент-Маргерит — крупнейший из Лерински...


### Очистка данных

In [25]:
import unicodedata
class DataCleaner:
    def __init__(self) -> None:
        self.count_removed_symbols = dict()
        self.count_replaced_symbols = dict()
        self.dict_replaced_symbols = dict()

    # функция подсчета количества удаленных символов
    def addRemovedSymbol(self, c):
        if not self.count_removed_symbols.keys().__contains__(c):
            self.count_removed_symbols[c] = 0
        self.count_removed_symbols[c] = self.count_removed_symbols[c] + 1

    def addReplacedSymbol(self, c):
        if not self.count_replaced_symbols.keys().__contains__(c):
            self.count_replaced_symbols[c] = 0
        self.count_replaced_symbols[c] = self.count_replaced_symbols[c] + 1

    # удаление знаков ударения и прочих символов unicode
    def unicodeToAscii(self, s):
        tmp = []
        for c in unicodedata.normalize('NFD', s):
            if unicodedata.category(c) != 'Mn':
                tmp.append(c)
            else:
                self.addRemovedSymbol(c)
        return ''.join(tmp)

    # если нужно удалить, то заменяем на пробел чтоб не потерят разделения слов
    def replaceChar(self, s):
        tmp = []
        char_to_remove = ['«', '»', '—', ',', '.']
        for i, c in enumerate(s):
            if char_to_remove.__contains__(c):
                self.addReplacedSymbol(c, s[i])
            else:
                tmp.append(c)

            if c == 'ё':
                s[i] = 'е'
                self.addReplacedSymbol(c)
        return s

    def removeChar(self, s):
        
        
        for c in s:
            
        return ''.join(tmp)

    def clean(self, df, column):
        for i in range(len(df)):
            df[column][i] = self.unicodeToAscii(df[column][i])
            df[column][i] = df[column][i].lower()
            df[column][i] = self.removeChar(df[column][i])
            df[column][i] = self.replaceChar(df[column][i])
        return df

    def summary(self):
        print("Data Cleaner log")
        print("\tRemoved Chars")
        dfRemoved = pd.DataFrame(["symbol", "count_removed"])
        for c in self.count_removed_symbols:
            s += f"\n\t'{c}' : {self.count_removed_symbols[c]}"
        s +=
        return s

In [26]:
t = DataCleaner()
df_train = t.clean(df_train, 'passage')
display(t.removed_symbols)
display(t.replaced_symbols)

C:\Users\leysh\AppData\Local\Temp\ipykernel_5388\3239124670.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column][i] = self.unicodeToAscii(df[column][i])
C:\Users\leysh\AppData\Local\Temp\ipykernel_5388\3239124670.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column][i] = df[column][i].lower()
C:\Users\leysh\AppData\Local\Temp\ipykernel_5388\3239124670.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column][

{'́': 708,
 '̆': 14138,
 '̈': 2843,
 '̔': 5,
 '̀': 2,
 '̣': 6,
 '̌': 1,
 '̥': 1,
 '̂': 2,
 '̊': 2,
 '̓': 3,
 '͂': 1,
 'ُ': 2}

{'«': 1059, '»': 1056, '—': 2076, '.': 10469, ',': 15453}

In [27]:
df_train['passage'][10]

'а бакингем-хоуз и ее коллеги суммировали последствия наиденные в обзорных статьях ранее частые случаи задержки роста плода результатом чего является укороченныи среднии срок беременности и сниженныи вес при рождении по сравнению с нормальными детьми дети 3-4-недельного возраста демонстрируют менее оптимальную  двигательную активность рефлексы и ориентацию в пространстве а дети 4-6 лет показывают низкии уровень работы неироповеденческих функции внимания эмоциональнои экспрессии и развития речи и языка величина этих влиянии часто небольшая частично в связи с независимыми переменными: включая употребление во время беременности алкоголя/табака а также факторы среды  у детеи школьного возраста проблемы с устоичивым вниманием и контролем своего поведения а также незначительные с ростом познавательными и языковыми способностями'

# Model

In [ ]:
train_path = data_dir / "DaNetQA/train.jsonl"
val_path = data_dir / "DaNetQA/val.jsonl"
test_path = data_dir / "DaNetQA/test.jsonl"

In [ ]:
from tfidf_baseline.DaNetQA import eval_DaNetQA

In [ ]:
_, DaNetQA_scores = eval_DaNetQA(train_path, val_path, test_path, vect)

In [ ]:
DaNetQA_scores["train"], DaNetQA_scores["val"]

In [ ]:
all_results["name"].append("DaNetQA")
all_results["train"].append(DaNetQA_scores["train"])
all_results["val"].append(DaNetQA_scores["val"])

In [ ]:
save_output(DaNetQA_scores["test_pred"], output_dir / "DaNetQA.jsonl")